In [94]:
import torch
import torch.nn as nn 
import os 
import sys
project_root = os.path.abspath("..")  # Adjust if needed

# Add the project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

from src.models.attentionVae import AttentionVAE
from proteinshake.datasets import ProteinLigandInterfaceDataset, AlphaFoldDataset, GeneOntologyDataset
from src.utils import data_utils as dtu
from src.dataset_classes.sequenceDataset import SequenceDataset
from torch.utils.data import DataLoader, Dataset, Subset
from src.utils.data_utils import *

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
dataset = ProteinLigandInterfaceDataset(root='../data').to_point().torch()

In [96]:
max_seq_length = 500
seq_dataset = SequenceDataset(dataset, max_seq_length, transformer_input=True, return_proteins = True)
seq_dataloader = DataLoader(seq_dataset, batch_size = 128, shuffle = False)

100%|██████████| 4642/4642 [00:01<00:00, 2636.13it/s]


In [97]:
torch.sum(seq_dataset.__getitem__(0)[1], dim = 1)[-84:]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [109]:
next(iter(seq_dataloader))[1].shape

torch.Size([128, 500, 21])

In [110]:
N_layers =  1
embed_dim = 256
hidden_dim = 256
num_heads = 2
dropout = 0.1
latent_dim = 2
seq_len = 500
if torch.cuda.is_available():
    torch.cuda.current_device()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

attention_vae_test = AttentionVAE(optimizer=torch.optim.Adam, 
                                  optimizer_param={'lr':0.001},
                                  N_layers=N_layers,
                                  embed_dim=embed_dim,
                                  hidden_dim=hidden_dim,
                                  num_heads=num_heads,
                                  dropout=dropout,
                                  latent_dim=latent_dim,
                                  seq_len=seq_len)

In [111]:
test_in = next(iter(seq_dataloader))

In [112]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [113]:
attention_vae_test.get_attention_weights = True
attention_vae_test.pos_encoder.device = device

In [114]:
device

device(type='cpu')

In [115]:
test_out = attention_vae_test(test_in[0])

In [116]:
test_in[1].shape

torch.Size([128, 500, 21])

In [93]:
test_out[4].shape

torch.Size([128, 500, 21])

In [50]:
amino_encoding_dict = {'S': 0,'Y': 1,'F': 2,'K': 3,'V': 4,'C': 5,
                            'A': 6,'H': 7,'T': 8,'I': 9,'W': 10,'R': 11,
                            'P': 12,'L': 13,'G': 14,'N': 15,'E': 16,
                            'Q': 17,'M': 18,'D': 19, 'X':20}

In [44]:
k = next(iter(seq_dataloader))

In [23]:
k

NameError: name 'k' is not defined

In [51]:
embedding_layer = nn.Embedding(21,256, padding_idx=20)

In [72]:
sum(k[0] == 20)

tensor(84)

In [75]:
embedding_layer(k).shape

torch.Size([128, 500, 256])

In [32]:
attention_vae_test.attention_weights[0].shape

torch.Size([128, 500, 500])

In [36]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping
import pytorch_lightning as pl
optimizer = torch.optim.Adam
optimizer_param = {'lr':0.001}
trainer = pl.Trainer(max_epochs=100,
    accelerator="auto",
    devices=1,
    logger=TensorBoardLogger(save_dir="logs/"))

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [37]:
trainer.fit(attention_vae_test, seq_dataloader)

/opt/anaconda3/envs/ProteinManifoldLearning/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/opt/anaconda3/envs/ProteinManifoldLearning/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

  | Name                        | Type               | Params | Mode 
---------------------------------------------------------------------------
0 | pos_encoder                 | PositionalEncoding | 0      | train
1 | attention_encoders          | ModuleList         | 12.3 K | train
2 | global_forward_layer        | Linear             | 125 K  | train
3 | forward_latent_mean_layer   | Linear             | 1.3 M  | train
4 | forward_latent_logvar_layer | Linear             | 1.3 M  | train
5 | soft                        | Softmax            | 0      | tr

Training: |          | 0/? [00:00<?, ?it/s]

mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0
mps:0



Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined